In [2]:
from experiments.model_training.train_qwen import train

/home/ubuntu/miniconda3/envs/tau2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-05 11:55:35.502 | INFO     | tau2.utils.utils:<module>:27 - Using data directory from source: /lambda/nfs/victor-north-tx/tau2-bench-private/data
2025-08-05 11:55:36.722 | INFO     | tau2.utils.llm_utils:<module>:43 - VLLM_API_BASE: http://127.0.0.1:8000/v1
2025-08-05 11:55:36.722 | INFO     | tau2.utils.llm_utils:<module>:44 - OLLAMA_API_BASE: http://127.0.0.1:11434
2025-08-05 11:55:36.723 | INFO     | tau2.utils.llm_utils:<module>:78 - LiteLLM: Cache is disabled
2025-08-05 11:55:36.723 | WARNING  | tau2.utils.llm_utils:<module>:85 - Sonnet thinking is disabled
2025-08-05 11:55:36.797 | DEBUG    | tau2.registry:<module>:194 - Registering default components...
2025-08-05 11:55:36.798 | DEBUG    | tau2.registry:<mod

In [ ]:
from pathlib import Path

model_name = "Qwen2.5-3B-instruct"
qwen_model = f"/home/ubuntu/victor-north-tx/models/{model_name}"
train_dataset_path = "data/train_full-v1.jsonl"
test_dataset_path = "data/test_full-v1.jsonl"
trained_model_name = f"{model_name}-sft-full-tau2-v1"
output_dir = f"./data/{trained_model_name}"
report_to_wandb = True
run_name = f"{trained_model_name}"
if Path(output_dir).exists():
    raise ValueError(f"Output directory {output_dir} already exists. Please delete it or choose a different output directory.")

max_train_datapoints = None
max_test_datapoints = None
use_peft = False
patience = 2

train(
    qwen_model=qwen_model, 
    train_dataset_path=train_dataset_path, 
    test_dataset_path=test_dataset_path, 
    output_dir=output_dir, 
    max_train_datapoints=max_train_datapoints, 
    max_test_datapoints=max_test_datapoints, 
    use_peft=use_peft,
    patience=patience,
    report_to_wandb=report_to_wandb,
    run_name=run_name
    )

Train dataset length: 1722
Test dataset length: 1107
Loading tokenizer and model for /home/ubuntu/victor-north-tx/models/Qwen2.5-3B-instruct
Tokenizer has a chat template.
Tokenizer eos_token_id: 151645, pad_token_id: 151643


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.22s/it]


Model loaded in 12.00 seconds


Truncating eval dataset: 100%|██████████| 1107/1107 [00:00<00:00, 9212.11 examples/s] 
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,0.001300,0.001436
2,0.001400,0.001415
3,0.001500,0.001359


In [1]:
from experiments.model_training.train_qwen import get_model_and_tokenizer, load_dataset
model, tokenizer = get_model_and_tokenizer(model_name=output_dir)

/home/ubuntu/miniconda3/envs/tau2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-05 13:43:48.299 | INFO     | tau2.utils.utils:<module>:27 - Using data directory from source: /lambda/nfs/victor-north-tx/tau2-bench-private/data
2025-08-05 13:43:49.528 | INFO     | tau2.utils.llm_utils:<module>:43 - VLLM_API_BASE: http://127.0.0.1:8000/v1
2025-08-05 13:43:49.529 | INFO     | tau2.utils.llm_utils:<module>:44 - OLLAMA_API_BASE: http://127.0.0.1:11434
2025-08-05 13:43:49.529 | INFO     | tau2.utils.llm_utils:<module>:78 - LiteLLM: Cache is disabled
2025-08-05 13:43:49.530 | WARNING  | tau2.utils.llm_utils:<module>:85 - Sonnet thinking is disabled
2025-08-05 13:43:49.603 | DEBUG    | tau2.registry:<module>:194 - Registering default components...
2025-08-05 13:43:49.604 | DEBUG    | tau2.registry:<mod

NameError: name 'output_dir' is not defined

In [14]:
from experiments.model_training.train_qwen import get_model_and_tokenizer, QWEN25_TEMPLATE, check_chat_template_supports_assistant_only_loss, load_dataset

with open('test.jinja', mode='r') as fp:
    QWEN25_TEMPLATE = fp.read()

print(QWEN25_TEMPLATE)

model_name = "Qwen2.5-3B-instruct"
qwen_model = f"/home/ubuntu/victor-north-tx/models/{model_name}"
tokenizer = get_model_and_tokenizer(qwen_model)[1]
tokenizer.chat_template = QWEN25_TEMPLATE
tokenizer.eos_token
tokenizer.eos_token_id
tokenizer.pad_token_id
tokenizer.chat_template

print(check_chat_template_supports_assistant_only_loss(tokenizer))

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.13s/it]

Model loaded in 4.63 seconds
True


In [15]:
from experiments.model_training.train_qwen import DEFAULT_TRAIN_DATASET_PATH, DEFAULT_TEST_DATASET_PATH, DEFAULT_MAX_TRAIN_DATAPOINTS, DEFAULT_MAX_TEST_DATAPOINTS, load_dataset

train_dataset = load_dataset(DEFAULT_TRAIN_DATASET_PATH, max_datapoints=DEFAULT_MAX_TRAIN_DATAPOINTS)
test_dataset = load_dataset(DEFAULT_TEST_DATASET_PATH, max_datapoints=DEFAULT_MAX_TEST_DATAPOINTS)


In [16]:
train_dataset[0]["messages"][1]

{'content': 'Hi! How can I help you today?',
 'role': 'assistant',
 'tool_call_id': None,
 'tool_calls': None}

In [19]:
res = tokenizer.apply_chat_template(train_dataset[0]["messages"])
# print(res)

In [20]:
res = tokenizer.apply_chat_template(train_dataset[0]["messages"], tokenize=True, return_dict=True, add_generation_prompt=False, return_assistant_tokens_mask=True)
print(res)

{'input_ids': [151644, 8948, 198, 27, 62295, 397, 2610, 525, 264, 6002, 2473, 8315, 429, 8609, 279, 1196, 4092, 311, 279, 366, 34790, 29, 3897, 3685, 624, 16014, 1817, 2484, 498, 646, 2987, 510, 12, 11000, 264, 1943, 311, 279, 1196, 624, 12, 7405, 264, 5392, 1618, 624, 98743, 25, 1446, 4157, 653, 2176, 518, 279, 1852, 882, 50347, 2679, 498, 3624, 1467, 2213, 1393, 3259, 264, 5392, 1618, 11, 279, 8315, 686, 4828, 458, 1465, 624, 1178, 2213, 686, 387, 3208, 311, 279, 1196, 13, 3155, 537, 990, 419, 2070, 369, 697, 1828, 32711, 382, 21453, 311, 387, 10950, 323, 2677, 1795, 279, 4842, 13, 23240, 1281, 2704, 498, 6923, 2697, 4718, 1172, 624, 522, 62295, 397, 7926, 8018, 397, 2, 6553, 1056, 20713, 10974, 271, 785, 1482, 882, 374, 220, 17, 15, 17, 19, 12, 15, 20, 12, 16, 20, 220, 16, 20, 25, 15, 15, 25, 15, 15, 25877, 382, 2121, 458, 32475, 8315, 11, 498, 646, 1492, 3847, 3070, 2190, 97219, 3070, 34053, 97219, 476, 3070, 18515, 334, 10971, 40062, 13, 1446, 1083, 3705, 3070, 1097, 42950, 323, 1

In [ ]:
# QWEN25_TEMPLATE = "{%- if tools %}\n    {{- '<|im_start|>system\\n' }}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- messages[0]['content'] }}\n    {%- else %}\n        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}\n    {%- endif %}\n    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}\n    {%- for tool in tools %}\n        {{- \"\\n\" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}\n    {%- else %}\n        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}\n    {%- endif %}\n{%- endif %}\n{%- for message in messages %}\n    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) %}\n        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}\n    {%- elif message.role == \"assistant\" %}\n       {% generation %}\n       {{- '<|im_start|>' + message.role }}\n        {%- if message.content %}\n            {{- '\\n' + message.content }}\n        {%- endif %}\n        {%- for tool_call in message.tool_calls %}\n            {%- if tool_call.function is defined %}\n                {%- set tool_call = tool_call.function %}\n            {%- endif %}\n            {{- '\\n<tool_call>\\n{\"name\": \"' }}\n            {{- tool_call.name }}\n            {{- '\", \"arguments\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- '}\\n</tool_call>' }}\n        {%- endfor %}\n        {{- '<|im_end|>\\n' }}\n       {% endgeneration %}\n    {%- elif message.role == \"tool\" %}\n        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}\n            {{- '<|im_start|>user' }}\n        {%- endif %}\n        {{- '\\n<tool_response>\\n' }}\n        {{- message.content }}\n        {{- '\\n</tool_response>' }}\n        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}\n            {{- '<|im_end|>\\n' }}\n        {%- endif %}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\\n' }}\n{%- endif %}\n"


{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."}
]

res = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, add_generation_prompt=False, return_assistant_tokens_mask=True)
print(res)
[tokenizer.decode(token_id) for i, token_id in enumerate(res['input_ids']) if res['assistant_masks'][i]]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 3838, 374, 279, 6722, 315, 9625, 30, 151645, 198, 151644, 77091, 198, 785, 6722, 315, 9625, 374, 12095, 13, 151645, 271], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['<|im_start|>',
 'assistant',
 '\n',
 'The',
 ' capital',
 ' of',
 ' France',
 ' is',
 ' Paris',
 '.',
 '<|im_end|>',
 '\n\n']